# Florida Panthers Feature Selection
* StellarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* October 15, 2021

## Hypothesis
Write about the hunch you have and why you're running this experiment.

## Experiment
Document the experiment including selecting data, data transformations, feature engineering and modelling

In [1]:
import getpass
import pyodbc
import pandas as pd
import warnings

from pycaret.classification import *

warnings.filterwarnings('ignore')

### Let's connect to MSSQL and run a stored proc to get our dataset:

In [6]:
SERVER = '34.206.73.189' 
DATABASE = 'datascience' 
USERNAME = 'dsAdminWrite' 
PASSWORD = getpass.getpass(prompt='Enter your password')
CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)

Enter your password ··················


In [7]:
lkupclientid = 93 # Florida Panthers

cursor = CNXN.cursor()

storedProc = (
    f"""Exec [stlrNHLPanthers].[ds].[getRetentionScoringModelData] {lkupclientid}"""
)

df = pd.read_sql(storedProc, CNXN)

CNXN.commit()
cursor.close()

df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14067 entries, 0 to 14066
Data columns (total 55 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lkupClientId              14067 non-null  int64  
 1   dimCustomerMasterId       14067 non-null  int64  
 2   customerNumber            14067 non-null  object 
 3   year                      14067 non-null  object 
 4   productGrouping           14067 non-null  object 
 5   totalSpent                14067 non-null  float64
 6   recentDate                14067 non-null  object 
 7   attendancePercent         14067 non-null  float64
 8   renewedBeforeDays         14067 non-null  int64  
 9   isBuyer                   14067 non-null  object 
 10  source_tenure             14067 non-null  int64  
 11  tenure                    14067 non-null  int64  
 12  distToVenue               0 non-null      object 
 13  totalGames                14067 non-null  int64  
 14  recenc

### Let's drop the features that have lots of null values:

In [4]:
df.drop([
    'distToVenue', 
    'lengthOfResidenceInYrs',
    'annualHHIncome',
    'education',
    'urbanicity',
    'isnextyear_buyer',
    'isnextyear_samepkg_buyer',
    'pkgupgrade_status',
    'auto_renewal'],
    axis=1, 
    inplace=True
)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15391 entries, 0 to 15390
Data columns (total 46 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   lkupClientId          15391 non-null  int64  
 1   dimCustomerMasterId   15391 non-null  int64  
 2   customerNumber        15391 non-null  object 
 3   year                  15391 non-null  object 
 4   productGrouping       15391 non-null  object 
 5   totalSpent            15391 non-null  float64
 6   recentDate            15391 non-null  object 
 7   attendancePercent     15391 non-null  float64
 8   renewedBeforeDays     15391 non-null  int64  
 9   isBuyer               15391 non-null  object 
 10  source_tenure         15391 non-null  int64  
 11  tenure                15391 non-null  int64  
 12  totalGames            15391 non-null  int64  
 13  recency               15391 non-null  int64  
 14  missed_games_1        15391 non-null  int64  
 15  missed_games_2     

### In order to compare two sets of features, we need to create some datasets for training and evalution:

In [8]:
df_train_A = df.sample(frac=0.9, random_state=786)
df_train_B = df.sample(frac=0.9, random_state=786)

df_eval_A = df.drop(df_train_A.index)
df_eval_B = df.drop(df_train_B.index)

print('Data for Modeling (A Class): ' + str(df_train_A.shape))
print('Unseen Data For Predictions: ' + str(df_eval_A.shape))

print('Data for Modeling (A Class): ' + str(df_train_A.shape))
print('Unseen Data For Predictions: ' + str(df_eval_A.shape))

Data for Modeling (A Class): (12660, 55)
Unseen Data For Predictions: (1407, 55)
Data for Modeling (A Class): (12660, 55)
Unseen Data For Predictions: (1407, 55)


### In order to compare two sets of features, we need to create some datasets for training and evalution:## Observations
Here you can document some ideas on the results from above

## Conclusions
Here you can talk about next steps, did the experiment work? If yes, what to do next? If no, why?